In [2]:
import argparse
from enum import Enum
import sys
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException

import time
import random

In [3]:
driver = webdriver.Firefox()

The geckodriver version (0.35.0) detected in PATH at C:\Users\alejandro.regueiroru\.wdm\drivers\geckodriver\win64\v0.35.0\geckodriver.exe might not be compatible with the detected firefox version (138.0.4); currently, geckodriver 0.36.0 is recommended for firefox 138.*, so it is advised to delete the driver in PATH and retry


In [4]:
def waitFinishLoad(driver):
    time.sleep(0.1)

In [5]:
def clickByXPath(auxText, driver):
    waitFinishLoad(driver)
    elem = driver.find_element(By.XPATH, auxText)
    elem.click()
    waitFinishLoad(driver)

In [6]:
Global_inflation_path = "https://fred.stlouisfed.org/tags/series?t=inflation;world+bank"

In [7]:
driver.get(Global_inflation_path)

In [8]:
try:
    link_list = []
    while True:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "titleLink")))
        link_elements = driver.find_elements(By.ID, "titleLink")
        for link in link_elements:
            link_list.append(link.get_attribute("href"))
        next_page = driver.find_element(By.XPATH, ".//a[@title='next page']")
        next_page = next_page.get_attribute('href')
        driver.get(next_page)
except Exception as e:
    print(f"Error: {e}")

print("Fin enlaces")

Error: Message: Unable to locate element: .//a[@title='next page']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:552:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Fin enlaces


In [9]:
link_list

['https://fred.stlouisfed.org/series/FPCPITOTLZGUSA',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGJPN',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGGBR',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGDEU',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGCHN',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGCAN',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGIND',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGWLD',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGBRA',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGITA',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGTUR',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGFRA',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGEMU',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGARG',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGMEX',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGAUS',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGKOR',
 'https://fred.stlouisfed.org/series/FPCPITOTLZGEUU',
 'https://fred.stlouisfed.or

In [10]:
import pandas as pd

In [11]:
# Lista para almacenar los datos extraídos
data_list = []

# Recorremos la lista de links y dentro de cada link recogemos todos los elementos con la clase "highcharts-point" y extraemos el aria-label
for link in link_list:
    driver.get(link)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "highcharts-point")))
    try:
        # Extraer el título de la página para obtener el nombre del país
        page_title = driver.title
        if "Inflation, consumer prices for " in page_title:
            country = page_title.split("Inflation, consumer prices for ")[-1].split("(")[0].strip()
        else:
            country = page_title.split("Inflation, consumer prices: All Income Levels for ")[-1].split("(")[0].strip()

        elems = driver.find_elements(By.CLASS_NAME, "highcharts-point")
        print(f"Found {len(elems)} highcharts-point elements in {link}")
        for elem in elems:
            aria_label = elem.get_attribute("aria-label")
            # El aria-label tiene el formato año, porcentaje
            # Separamos el año y el porcentaje
            year, percentage = aria_label.split(", ")
            # Limpiamos el año y el porcentaje
            year = year.strip()
            # En percentage nos quedamos solo con el número y no con el " Percent" que viene
            percentage = percentage.split(" ")[0]
            percentage = percentage.strip()
            # Agregamos los datos a la lista
            data_list.append({"Country": country, "Year": year, "Percentage": percentage})

    except NoSuchElementException:
        print(f"No highcharts-point found in {link}")

# Crear un DataFrame de pandas con los datos extraídos
df = pd.DataFrame(data_list)


Found 65 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGUSA
Found 65 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGJPN
Found 65 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGGBR
Found 65 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGDEU
Found 38 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGCHN
Found 65 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGCAN
Found 65 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGIND
Found 43 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGWLD
Found 44 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGBRA
Found 65 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGITA
Found 65 highcharts-point elements in https://fred.stlouisfed.org/series/FPCPITOTLZGTUR
Found 65 highcharts-point elemen

In [14]:
# Cerrar el driver
driver.quit()

In [12]:
df

,Country,Year,Percentage
0,the United States,1960,1.45798
1,the United States,1961,1.07072
2,the United States,1962,1.19877
3,the United States,1963,1.23967
4,the United States,1964,1.27891
...,...,...,...
10342,Tonga,2020,-0.34974
10343,Tonga,2021,5.64051
10344,Tonga,2022,10.97137
10345,Tonga,2023,6.35113


In [ ]:
df.to_json('../json/inflation.json', orient='records', indent=4)